In [1]:
%cd ..

/home/teo/userdata/virtuosoNet


In [27]:
from virtuoso.train import prepare_dataloader
from virtuoso.parser import get_parser
from virtuoso import utils, model as modelzoo
from virtuoso.dataset import ScorePerformDataset, FeatureCollate
from virtuoso.module import ContextAttention
from virtuoso.model_utils import make_higher_node
import torch

In [7]:
parser = get_parser()
args = parser.parse_args(
    args=["--yml_path=ymls/han_measnote.yml",
          "--data_path=datasets/main_dataset_clamped",
          "--emotion_data_path=datasets/emotion_dataset_clamped",
          "--device=cpu"]
)
args, net_params, configs = utils.handle_args(args)
device = 'cpu'

In [9]:
train_loader, valid_loader, emotion_loader, multi_perf_loader = prepare_dataloader(args)

In [32]:
attention = ContextAttention(16, 2).to('cuda')

In [36]:
from tqdm import tqdm

for i in tqdm(range(100)):
  train_loader.dataset.update_slice_info()
  for batch in train_loader:
    note_locations = batch[4]
    dummy_note = torch.ones(note_locations['beat'].shape[0], note_locations['beat'].shape[1], 16).to('cuda')
    beat = make_higher_node(dummy_note, attention, note_locations['beat'], note_locations['beat'], lower_is_note=True)
    measure = make_higher_node(beat, attention, note_locations['beat'], note_locations['measure'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [36:32<00:00, 21.92s/it]


In [37]:
dummy_note.shape

torch.Size([27, 823, 16])